In [ ]:
import json
import math
import networkx as nx
import pandas as pd
import altair as alt
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import distfit
import os

from generator_v2 import Generator

In [ ]:
alt.data_transformers.enable("vegafusion")

In [ ]:
COLOR = "#99d8c9"

In [ ]:
alt.__version__

In [ ]:
# Set the ggplot2 theme
alt.themes.enable("ggplot2")

In [ ]:
# Essayer avec 1000 pour voir max(n1, n2) / n1 + n2"
N = 1000
# N_edges = N // 2
# Tester plusierus valeurs d'aretes (50, 100, 200), pour la fraction des hyperaretes
N_edges = 200

# Last value is the one used for most simulations
# N_EDGES = [50, 100, 200, 500, 300]
# N_EDGES = [300]
# N_EDGES = [100, 200, 300, 500]
Ns = [200, 300, 500, 700, 1000]


N_coms = 4
sampling_strat = "weighted"
# sampling_strat = "max"
p = 30/N
q = 3/N

In [ ]:
community_array = [0 for x in range(N//2)]  + [1 for x in range(N//2)]

In [ ]:
len(community_array)

# Node Degree Multi Simulation

In [ ]:
N_sim = 20

In [ ]:
df_sim = pd.DataFrame(columns=["count", "simNumber"], dtype=int)
for i in range(N_sim):
    gen = Generator(N, N // 2, N_coms, 20/N, 3/N, community_array, sampling_strat)
    gen.run()
    degrees = dict(gen.degrees()).values()
    degrees_df = pd.DataFrame(degrees, columns=["degree"])

    countdf = degrees_df.groupby(['degree'])['degree'].count()
    countdf = countdf.to_frame().rename(columns={"degree": "count"})
    countdf["simNumber"] = i
    
    df_sim = pd.concat([df_sim, countdf])

In [ ]:
df_sim = df_sim.reset_index(names="degree")
# df_sim.rename(columns={}

In [ ]:
df_sim

In [ ]:
bars = alt.Chart(df_sim).mark_bar(color=COLOR).encode(
    alt.X("degree:Q", scale=alt.Scale(domain=[0, 18])),
    alt.Y("mean(count):Q"),
)

In [ ]:
error = alt.Chart(df_sim).mark_errorbar(extent="ci", rule=True).encode(
    x=alt.X("degree:Q", scale=alt.Scale(domain=[0, 18])),
    y=alt.Y(
        "count:Q",
        scale=alt.Scale(zero=False),
        title="Absolute Frequency"
    ),
)

In [ ]:
bars + error

# Hyperedge Size

## Hyperedge Sim

In [ ]:
df_sim = pd.DataFrame(columns=["count", "simNumber"], dtype=int)
for i in range(N_sim):
    gen = Generator(N, N_edges, N_coms, p, q, community_array, sampling_strat)
    gen.run()
    hsizes = dict(gen.hyperedge_sizes()).values()
    hsizes_df = pd.DataFrame(hsizes, columns=["hsize"])

    countdf = hsizes_df.groupby(['hsize'])['hsize'].count()
    countdf = countdf.to_frame().rename(columns={"hsize": "count"})
    countdf["simNumber"] = i
    
    df_sim = pd.concat([df_sim, countdf])

In [ ]:
df_sim = df_sim.reset_index(names="hsize")

In [ ]:
df_sim.head()

In [ ]:
bars = alt.Chart(df_sim).mark_bar(color=COLOR).encode(
    alt.X("hsize:Q", scale=alt.Scale(domain=[0, 25]), title="Hyperedge Size"),
    alt.Y("mean(count):Q"),
)

error = alt.Chart(df_sim).mark_errorbar(extent="ci", rule=True).encode(
    x=alt.X("hsize:Q", scale=alt.Scale(domain=[0, 25])),
    y=alt.Y(
        "count:Q",
        scale=alt.Scale(zero=False),
        title="Absolute Frequency"
    ),
)

In [ ]:
bars + error

# Fraction Dist

In [ ]:
# Faire pareil sur 100/1000 réseaux
# Accumuler la liste des degrées/hsize
# Une value de p/q => plusieurs simulations => faire le fit la dessus

In [ ]:
p_init = 30 / N
q_init = 30 / N

In [ ]:
q_init

In [ ]:
def fit_function(x, n, p):
    return stats.binom.pmf(x, n, p)

In [ ]:
qs = np.arange(0, 1.01, 0.10)
# qs = np.append(qs, [0.95, 0.99])
qs = np.append(qs, [0.01, 0.05])

In [ ]:
qs

In [ ]:
fits_N = 1000
fits_strat = "weighted"

In [ ]:
p = p_init
q = q_init

df_degrees = pd.DataFrame(columns=["degree"], dtype=int)
df_hsizes = pd.DataFrame(columns=["hsize"], dtype=int)

df_sim = pd.DataFrame(columns=["sim", "type", "count"], dtype=int)
df_fraction = pd.DataFrame(columns=["sim", "count", "fraction0"], dtype=int)
df_fraction2 = pd.DataFrame(columns=["sim", "value"])

# df_peff = pd.DataFrame(columns=["peff", "p", "q", "q_frac"], dtype=int)

df_fits = pd.DataFrame(columns=["peff", "Neff", "p_hsize_eff", "n_eff_hsize", "p", "q", "q_frac"], dtype=int)

# Same as df_fits but other format so easier altair plots 
df_fits2 = pd.DataFrame()

increment = 0.05
N_sim = int(1 / increment) + 1
q_frac_order = []

# Prendre 100 configs
# N_graphs = 5
# N_graphs = 50
N_graphs = 100
# N_graphs = 3

q_to_degreefit = {}
q_to_hsizefit = {}

# for i in range(N_sim):
for q_frac in qs:
    # q = round(q_init - (p * increment * i), 4)
    q = round(p * q_frac,  4)
    print(p, q)
    
    # q_frac = f"{round(1 - (increment * (i)), 3)}p"
    q_frac_order.append(q_frac) 

    for strat in ["weighted", "max", "min", "frequent"]:
        # for n in Ns:
        for n in [2000]:
            all_degrees = []
            all_hsizes = []  
    
            community_array = []
            for i in range(N_coms):
                community_array += [i for x in range(n // N_coms)]
            
            for n_graph in range(N_graphs):
                # gen = Generator(N, N_edges, N_coms, p, q, community_array, sampling_strat)
                gen = Generator(n, N_edges, N_coms, p, q, community_array, strat)
                gen.run()
                
                # comp = gen.hyperedges_types()
                # n_pure = comp.count("pure")
                # n_mixed = comp.count("mixed")
                # df = pd.DataFrame({"sim": [i, i], "q": [q_frac, q_frac], "type": ["pure", "mixed"], "count": [n_pure, n_mixed]})
                # df_sim = pd.concat([df_sim, df])
            
            #     For fraction distribution of mixed edges
                # comp = gen.mixed_he_fraction_to_count()
                # for fraction, count in comp.items():
                #     df = pd.DataFrame({"sim": [i], "q": [q_frac], "count": [count], "fraction0": fraction})
                #     df_fraction = pd.concat([df_fraction, df])
                    
                #     For com distrib of hyperedges
                comp = gen.hyperedges_nmax()
                ginis = gen.ginis()
                
                # for fraction in comp:
                df = pd.DataFrame({"q": [q] * len(comp), "value": comp, "gini": ginis, "N_nodes": [n] * len(comp), "strat": [strat] * len(comp)})
                df_fraction2 = pd.concat([df_fraction2, df])
                
                # for peffectif computation
                if n == fits_N and strat == fits_strat:
                    degrees = dict(gen.degrees()).values()
                    hsizes = dict(gen.hyperedge_sizes()).values()
                    
                    all_degrees = all_degrees + list(degrees)
                    all_hsizes = all_hsizes + list(hsizes)
        

            # DO THE DISTRIBUTIONS AND FIT ONLY FOR THOSE VALUES
            if n == fits_N and strat == fits_strat:
                df = pd.DataFrame({"degree": all_degrees, "q": q_frac, "q / p": q / p, "N_nodes": n})
                df_degrees = pd.concat([df_degrees, df])
                                      
                df = pd.DataFrame({"hsize": all_hsizes, "q": q_frac, "q / p": q / p, "N_nodes": n})
                df_hsizes = pd.concat([df_hsizes, df])
    
                # Distrib of degree
                # Bounds of n and p effectif
                bounds = [(0, N_edges), (0, p)]
                
                dist = stats.binom
                res = stats.fit(dist, all_degrees, bounds)
                
                peff = res.params[1]
                neff = res.params[0]
                
                bounds2 = [(N_edges, N_edges), (0, p)]
                res = stats.fit(dist, all_degrees, bounds2)
                peffNfixed = res.params[1]
                neffNfixed = res.params[0]
                
                bounds3 = [(0, N_edges), (p, p)]
                res = stats.fit(dist, all_degrees, bounds3)
                peffpfixed = res.params[1]
                neffpfixed = res.params[0]
                
                df_notfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff], "Neff": [neff], "bounds": ["not fixed"], "measure": ["degree"]})
                df_pfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peffNfixed], "Neff": [neffNfixed], "bounds": ["n Fixed"], "measure": ["degree"]})
                df_nfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peffpfixed], "Neff": [neffpfixed], "bounds": ["p Fixed"], "measure": ["degree"]})
                df_fits2 = pd.concat([df_fits2, df_notfixed, df_pfixed, df_nfixed])
                
                q_to_degreefit[q] = res
                # print(res.nllf(), res.success)
                # print(neff, peff)
                
                # mean = np.mean(all_degrees)
                # variance = np.var(all_degrees)
                
                # pForm = 1 - (variance / mean)
                # NForm = mean / (1 -  (variance / mean))
                # print("compute", NForm, pForm)
                
                # Distrib of hsizes
                dist = stats.binom
                
                # bounds = [(N / N_coms, N), (q, p)]
                bounds = [(N / N_coms, N), (0, p)]
                res = stats.fit(dist, all_hsizes, bounds)
                peffhsize = res.params[1]
                neff_hsize = res.params[0]
                q_to_hsizefit[q] = res
                
                bounds = [(N / N_coms, N), (p, p)]
                res = stats.fit(dist, all_hsizes, bounds)
                peff_hsize_pfixed = res.params[1]
                neff_hsize_pfixed = res.params[0]
                
                bounds = [(N, N), (0, p)]
                res = stats.fit(dist, all_hsizes, bounds)
                peff_hsize_nfixed = res.params[1]
                neff_hsize_nfixed = res.params[0]
                
                df = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff], "Neff": [neff], "peffpfixed": [peffpfixed], "Neffpfixed": [neffpfixed], "peffnfixed": [peffNfixed], "Neffnfixed": [neffNfixed], "p_hsize_eff": [peffhsize], "n_eff_hsize": [neff_hsize]})
                df_fits = pd.concat([df_fits, df])
                # df_peff = pd.concat([df_peff, df])
                
                df_notfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peffhsize], "Neff": [neff_hsize], "bounds": ["not fixed"], "measure": ["hsize"]})
                df_pfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff_hsize_nfixed], "Neff": [neff_hsize_nfixed], "bounds": ["n Fixed"], "measure": ["hsize"]})
                df_nfixed = pd.DataFrame({"p": [p], "q": [q], "q_frac": [q_frac], "peff": [peff_hsize_pfixed], "Neff": [neff_hsize_pfixed], "bounds": ["p Fixed"], "measure": ["hsize"]})
                df_fits2 = pd.concat([df_fits2, df_notfixed, df_pfixed, df_nfixed])

In [ ]:
df_fraction2 = final

In [ ]:
fn = "he_distrib.csv"
if not os.path.exists(fn):
    df_fraction2.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_fraction2 = pd.read_csv(fn)

In [ ]:
fn = f"hsize_distrib_{fits_N}_{fits_strat}.csv"
if not os.path.exists(fn):
    df_hsizes.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_hsizes = pd.read_csv(fn)

In [ ]:
fn = f"degree_distrib_{fits_N}_{fits_strat}.csv"
if not os.path.exists(fn):
    df_degrees.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_degrees = pd.read_csv(fn)

In [ ]:
fn = f"fits1_{fits_N}_{fits_strat}.csv"
if not os.path.exists(fn):
    df_fits.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_fits = pd.read_csv(fn)

In [ ]:
fn = f"fits2_{fits_N}_{fits_strat}.csv"
if not os.path.exists(fn):
    df_fits2.to_csv(fn, sep=',', index=False, encoding='utf-8')
else:
    df_fits2 = pd.read_csv(fn)

## Degree histograms with fit

In [ ]:
df_degrees

In [ ]:
qs = [0.2, 0.4, 0.6, 0.8]
for q in qs:
    df = df_degrees[df_degrees["q"] == f"{q}p"]   
    fit = q_to_degreefit[q * p]
    
    chart = alt.Chart(df)

    # Create a density plot using transform_density
    density = chart.transform_density(
        density='degree',
        as_=['values', 'density'],  # Output field names
    ).mark_line(  # You can use mark_line() for a line plot
        opacity=0.6,  # Adjust the opacity of the area plot
    ).encode(
        x=alt.X('values:Q'),  # Set the X-axis label
        y=alt.Y('density:Q', title='Density'),  # Set the Y-axis label
    )
    
    
    # Calculate the binomial PDF values for a range of x values
    x_values = range(30)  # Assuming a range from 0 to 10
    binomial_pmf = [stats.binom.pmf(x, fit.params[0], fit.params[1]) for x in x_values]

    # Create a DataFrame for the binomial PDF
    binomial_df = pd.DataFrame({
        'values': x_values,
        'binomial_pmf': binomial_pmf
    })

    # Create a line plot for the binomial PDF
    binomial = alt.Chart(binomial_df).mark_line(color='red').encode(
        x=alt.X('values:Q', title="Node Degree"),
        y=alt.Y('binomial_pmf:Q'),
    )
    
    chart = density + binomial
    chart.display()

In [ ]:
df_fits

## Normalize

In [ ]:
for q in qs:
    q = round(q, 2)
    df = df_degrees[df_degrees["q"] == f"{q}p"]

    print(q, q*p, p)
    
    fit = q_to_degreefit[round(q * p, 3)]
    
    chart = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
    ).transform_calculate(
        pct='1 / datum.total'
    ).mark_bar(opacity=0.7).encode(
        alt.X('degree:O'),
        alt.Y('sum(pct):Q')
    )
    
    # Calculate the binomial PDF values for a range of x values
    x_values = range(19)  # Assuming a range from 0 to 10
    binomial_pmf = [stats.binom.pmf(x, fit.params[0], fit.params[1]) for x in x_values]

    fit_values = df_fits[df_fits["q_frac"] == f"{q}p"]
    binomial_pmf2 = [stats.binom.pmf(x, fit_values.Neffpfixed[0], fit_values.peffpfixed[0]) for x in x_values]
    binomial_pmf3 = [stats.binom.pmf(x, fit_values.Neffnfixed[0], fit_values.peffnfixed[0]) for x in x_values]

    # Create a DataFrame for the binomial PDF
    binomial_df = pd.DataFrame({
        'values': x_values,
        'binomial_pmf': binomial_pmf,
        'binomial_pmf2': binomial_pmf2,
        'binomial_pmf3': binomial_pmf3,
    })

    # Create a line plot for the binomial PDF
    # binomial = alt.Chart(binomial_df).mark_bar(color='red', opacity=0.3, width=2).encode(
    binomial = alt.Chart(binomial_df).mark_bar(color='red', opacity=0.3, width=2).encode(
        x=alt.X('values:O', title="Node Degree"),
        y=alt.Y('binomial_pmf:Q'),
    )

    binomial2 = alt.Chart(binomial_df).mark_bar(color='green', opacity=0.3, width=2, xOffset=6).encode(
        x=alt.X('values:O', title="Node Degree", bandPosition=0.2),
        y=alt.Y('binomial_pmf2:Q'),
    )

    # binomial3 = alt.Chart(binomial_df).mark_bar(color='purple', opacity=0.3, width=2, xOffset=-6).encode(
    #     x=alt.X('values:O', title="Node Degree", bandPosition=0.2),
    #     y=alt.Y('binomial_pmf3:Q'),
    # )
    binomial3 = alt.Chart(binomial_df).mark_line(color='purple', opacity=0.3, width=2).encode(
        x=alt.X('values:O', title="Node Degree"),
        y=alt.Y('binomial_pmf3:Q'),
    )
    
    # (chart + binomial + binomial2 + binomial3).display()

    # Only show the N fixed chart 
    (chart + binomial3).display()

## Group by

In [ ]:
df_degrees = df_degrees[df_degrees["N_nodes"] == 1000]

In [ ]:
df_degrees.head()

In [ ]:
df_fits = df_fits.round({'q_frac': 3})
df_fits.q_frac

In [ ]:
grouped = df_degrees.groupby('q / p')['degree'].value_counts(normalize=True).reset_index(name='Percentage')

In [ ]:
grouped = grouped.round({'q / p': 2})

In [ ]:
grouped['binomial'] = 0  # Initialize the column

for q in qs:
    q = round(q, 3)
    # print(q)
    df = df_degrees[df_degrees["q"] == f"{q}p"]
    # fit = q_to_degreefit[round(q * p, 3)]
    
    # Calculate the binomial PDF values for a range of x values
    x_values = range(19)  # Assuming a range from 0 to 10
    # binomial_pmf = [stats.binom.pmf(x, fit.params[0], fit.params[1]) for x in x_values]
    # fit_values = df_fits[df_fits["q_frac"] == f"{q}p"]
    # binomial_pmf2 = [stats.binom.pmf(x, fit_values.Neffpfixed[0], fit_values.peffpfixed[0]) for x in x_values]
    # binomial_pmf3 = [stats.binom.pmf(x, fit_values.Neffnfixed[0], fit_values.peffnfixed[0]) for x in x_values]

    # fit_values = df_fits[df_fits["q_frac"] == f"{q}p"]
    fit_values = df_fits[df_fits["q_frac"] == q]
    # print(fit_values.Neffpfixed)
    # print(fit_values.Neffpfixed.iloc[0])
    for x in x_values:
        binom_pdf = stats.binom.pmf(x, fit_values.Neffpfixed.iloc[0], fit_values.peffpfixed.iloc[0])
        # grouped.loc[(grouped['degree'] == x) & (grouped['q'] == f"{q}p"), 'binomial'] = binom_pdf
        grouped.loc[(grouped['degree'] == x) & (grouped['q / p'] == q), 'binomial'] = binom_pdf        

In [ ]:
grouped

In [ ]:
# grouped_selp_deg = grouped[grouped.q.isin(["0.0p", "0.3p", "0.7p", "1.0p"])]
grouped_selp_deg = grouped[grouped["q / p"].isin([0, 0.3, 0.7, 1])]

In [ ]:
grouped_selp_deg

In [ ]:
pdf = alt.Chart(grouped_selp_deg).mark_line(width=1, opacity=0.7, color="darkgrey").encode(
        x=alt.X('degree:O', title="Node Degree"),
        y=alt.Y('binomial:Q', title="Frequency"),
        # color="q"
        # facet=alt.Facet("q:O")    
)

In [ ]:
distrib = alt.Chart(grouped_selp_deg).mark_bar(width=5,  opacity=0.7, color="black").encode(
        x=alt.X('degree:O', title="Node Degree"),
        y=alt.Y('Percentage:Q'),
        # facet=alt.Facet("q:O")
        # color="q"
    )

In [ ]:
# alt.layer(pdf, distrib, data=grouped_selp_deg).facet(column='q')
alt.layer(pdf, distrib, data=grouped_selp_deg).facet(column='q / p')

## Hsize

### Group by

In [ ]:
df_hsizes = df_hsizes[df_hsizes["N_nodes"] == 1000]

In [ ]:
df_fits2 = df_fits2.round({'q_frac': 3})

In [ ]:
grouped = df_hsizes.groupby('q / p')['hsize'].value_counts(normalize=True).reset_index(name='Percentage')
grouped

In [ ]:
grouped = grouped.round({'q / p': 2})

In [ ]:
grouped['binomial'] = 0  # Initialize the column

for q in qs:
    q = round(q, 2)
    # df = df_hsizes[df_hsizes["q"] == f"{q}p"]
    df = df_hsizes[df_hsizes["q"] == q]
    
    # fit_values = df_fits2[(df_fits2["q_frac"] == f"{q}p") & (df_fits2["measure"] == "hsize") & (df_fits2["bounds"] == "p Fixed")]
    fit_values = df_fits2[(df_fits2["q_frac"] == q) & (df_fits2["measure"] == "hsize") & (df_fits2["bounds"] == "p Fixed")]
    
    x_values = range(60)  # Assuming a range from 0 to 10
    for x in x_values:
        binom_pdf = stats.binom.pmf(x, fit_values.Neff, fit_values.peff)
        grouped.loc[(grouped['hsize'] == x) & (grouped['q / p'] == q), 'binomial'] = binom_pdf

In [ ]:
# grouped_selp = grouped[grouped.q.isin(["0.0p", "0.3p", "0.7p", "1.0p"])]
grouped_selp = grouped[grouped["q / p"].isin([0, 0.3, 0.7, 1])]

In [ ]:
pdf = alt.Chart(grouped_selp).mark_line(width=1, opacity=0.7, color="darkgrey").encode(
        x=alt.X('hsize:O', title="Hyperedge Size", scale=alt.Scale(domain=list(range(3, 51)), padding=0.1, paddingInner=0.1, )),
        y=alt.Y('binomial:Q', title="Frequency"),
        # color="q"
        # facet=alt.Facet("q:O")    
)

In [ ]:
distrib = alt.Chart(grouped_selp).mark_bar(width=5,  opacity=0.7, color="black").encode(
        x=alt.X('hsize:O', title="Hyperedge Size", scale=alt.Scale(domain=list(range(3, 51)), padding=0.1, paddingInner=0.1)),
        y=alt.Y('Percentage:Q'),
        # facet=alt.Facet("q:O")
        # color="q"
    )

In [ ]:
alt.layer(pdf, distrib, data=grouped_selp).facet(column='q / p')

## Distribution of types of edges

In [ ]:
df_fraction

In [ ]:
alt.Chart(df_sim).mark_bar().encode(
    x=alt.X('q:O', sort=q_frac_order),
    y=alt.Y('count', title="Number of hyperedges"),
    color=alt.Color('type', 
    scale = alt.Scale(domain=['mixed', "pure"], range=['#9ebcda', '#e0ecf4']))
).properties(
    width=800,
    height=300
)

In [ ]:
alt.Chart(df_fraction).mark_bar().encode(
    x=alt.X('q:O', sort=q_frac_order),
    y=alt.Y('count'),
    color=alt.Color('fraction0').scale(scheme="lightgreyteal"),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'fraction0',
      sort='descending'
    )
    # scale = alt.Scale(domain=['mixed', "pure"], range=['#9ebcda', '#e0ecf4']))
).properties(
    width=800,
    height=300
)

## Hyperedge composition

In [ ]:
df_fraction2["q / p"] = df_fraction2["q"] / p
df_fraction2["N / E"] = df_fraction2["N_nodes"] / N_edges

In [ ]:
df_fraction_1000 = df_fraction2[df_fraction2["N_nodes"] == 1000]

In [ ]:
# Calculer la variance a la place, tester avec plusieurs valeurs de N, en faisant varié E, N = 200, 500, 1000

error = alt.Chart(df_fraction_1000).mark_errorband(extent="stdev").encode(
    x="q / p",
    y=alt.Y(
        "value:Q",
        # scale=alt.Scale(zero=False),
        # title="max(n1, n2) / n1 + n2",
        title="c"
    ),
)

mean_line = alt.Chart(df_fraction_1000).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "mean(value)",
        # title="Hyperedge Purity"
        # title="max(n1, n2) / n1 + n2"
        title="c"
    ),
)

error + mean_line

In [ ]:
error = alt.Chart(df_fraction_1000).mark_errorband(extent="stdev").encode(
    x="q / p",
    y=alt.Y(
        "gini:Q",
        # scale=alt.Scale(zero=False),
        # title="max(n1, n2) / n1 + n2",
        title="gini"
    ),
)

mean_line = alt.Chart(df_fraction_1000).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "mean(gini)",
        # title="Hyperedge Purity"
        # title="max(n1, n2) / n1 + n2"
        title="gini"
    ),
)

error + mean_line

In [ ]:
# alt.layer(pdf, distrib, data=grouped_selp_deg).facet(column='q')
alt.layer(error, mean_line, data=df_fraction_1000).facet(column='strat')

In [ ]:
alt.Chart(df_fraction2).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "mean(gini)",
        # title="Hyperedge Purity"
        # title="max(n1, n2) / n1 + n2"
        title="normalized gini"
    ),
    # color=alt.Color("N / E:N", scale=alt.Scale(scheme="category10"))
    # color=alt.Color("N / E:N", scale=alt.Scale()),
    strokeDash="N / E"
).facet(column='strat')

## Normalize 

In [ ]:
df_fraction2_var = (df_fraction2
    # .groupby('q / p')
    .groupby(['q / p', "N / E", "strat"])
    # ['value']
    ['gini']
    .agg(['mean', 'std'])
    .assign(stdNorm = lambda df: df['std'] / df['mean'])
    .reset_index())

In [ ]:
df_fraction2_var

In [ ]:
# Tracer en fct de E / N en faisant varier N
alt.Chart(df_fraction2_var).mark_line().encode(
    x="q / p",
    y=alt.Y(
        "stdNorm",
        # title="Hyperedge Purity"
        # title="max(n1, n2) / n1 + n2"
        title="std(gini) / mean(gini)",
    ),
    color=alt.Color("N / E", scale=alt.Scale(scheme="greys"))
).facet("strat")

In [ ]:
df_fraction2_max = (df_fraction2_var
    .groupby(["N / E", "strat"])
    ['stdNorm']
    .agg(['max'])
    # .assign(max = lambda df: df['std'] / df['mean'])
    .reset_index())

In [ ]:
df_fraction2_max

In [ ]:
# Tracer en fct de E / N en faisant varier N
alt.Chart(df_fraction2_max).mark_line().encode(
    x="N / E",
    y=alt.Y(
        "max",
        # title="Hyperedge Purity"
        # title="max(n1, n2) / n1 + n2"
        title="gini*",
    )
).facet("strat")

In [ ]:
# Pour quelques valeurs de q /p, tracer la distribution de la valeur
# Tracer la variance de la quantite vs q/p

In [ ]:
alt.Chart(df_fraction2[df_fraction2["q / p"] == 0.1]).transform_density(
    'value',
    as_=['value', 'density'],
).mark_line().encode(
    x="value:Q",
    y='density:Q',
)

In [ ]:
# Enlever 0. Bug dans le calcul de la densite.
# appeler la valeur de c
alt.Chart(df_fraction2[(df_fraction2.N_nodes == 1000) & (df_fraction2["q / p"] != 0)]).transform_density(
    'value',
    groupby=['q / p'],
    as_=['value', 'density'],
).mark_line().encode(
    x=alt.X("value:Q", title="c"),
    y='density:Q',
    color=alt.Color("q / p",  scale=alt.Scale(scheme="greys"))
)

In [ ]:
gini_plot = alt.Chart(df_fraction2[(df_fraction2.N_nodes == 1000) & (df_fraction2["q / p"].isno([0, 0.01]))]).transform_density(
    'gini',
    groupby=['q / p', "strat"],
    as_=['gini', 'density'],
).mark_line().encode(
    x=alt.X("gini:Q", title="gini"),
    y='density:Q',
    color=alt.Color("q / p",  scale=alt.Scale(scheme="greys"))
)

In [ ]:
gini_plot.facet("strat")

# Distrib effectives

In [ ]:
df_fits

## Degree distrib fit

In [ ]:
points = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff', title="p_eff"),
).properties(
    width=800,
    height=300
)

In [ ]:
regresssion = points.transform_regression('q', 'peff', method="poly").mark_line(color="black")

In [ ]:
line = alt.Chart(pd.DataFrame({'y': [p]})).mark_rule().encode(y='y')

In [ ]:
# points + regresssion + line
# Pour plusieurs valeur de N
points + line

### Normalize

In [ ]:
df_fits["peff / p"] = df_fits["peff"] / df_fits["p"] 
df_fits["p_h_eff / p"] = df_fits["p_hsize_eff"] / df_fits["p"] 
df_fits["q / p"] = df_fits["q"] / df_fits["p"] 

df_fits["Neff / Nedges"] = df_fits["Neff"] / N_edges
df_fits["NeffHsize / N"] = df_fits["n_eff_hsize"] / N

In [ ]:
df_fits2["q / p"] = df_fits2["q"] / df_fits2["p"] 
df_fits2["peff / p"] = df_fits2["peff"] / df_fits2["p"] 
df_fits2["neff / Nedges"] = df_fits2["Neff"] / N_edges
df_fits2["neff / N"] = df_fits2["Neff"] / N

## Old

In [ ]:
points = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff / p'),
).properties(
    width=800,
    height=300
)

points

In [ ]:
points_Neff = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('Neff / Nedges'),
).properties(
    width=800,
    height=300
)

points_Neff

In [ ]:
alt.Chart(df_fits).mark_line().encode(
    x=alt.X('peff / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('Neff / Nedges'),
).properties(
    width=800,
    height=300
)

In [ ]:
alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('Neffpfixed'),
).properties(
    width=800,
    height=300
)

In [ ]:
# Fitter par une droite (rouge)
# Point plus grand,

alt.Chart(df_fits2[df_fits2.measure == "degree"]).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / Nedges'),
    color="bounds"
).properties(
    width=800,
    height=300
)

In [ ]:
alt.Chart(df_fits2[df_fits2.measure == "degree"]).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff / p'),
    color="bounds"
).properties(
    width=800,
    height=300
)

In [ ]:
df_fits2

## Hsize fits

In [ ]:
points = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('p_h_eff / p'),
).properties(
    width=800,
    height=300
)

points

In [ ]:
points_Neff = alt.Chart(df_fits).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y("NeffHsize / N"),
).properties(
    width=800,
    height=300
)

points_Neff

## Several bounds

In [ ]:
chart = alt.Chart(df_fits2[df_fits2.measure == "hsize"]).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / N'),
    # color="bounds"
).properties(
    width=800,
    height=300
)

# chart

In [ ]:
chart + chart.transform_regression('q / p', 'neff / N').mark_line()

In [ ]:
alt.Chart(df_fits2[df_fits2.measure == "hsize"]).mark_point().encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('peff / p'),
    color="bounds"
).properties(
    width=800,
    height=300
)

In [ ]:
df = df_fits2[(df_fits2.measure == "hsize") & (df_fits2.bounds == "p Fixed")]
chart = alt.Chart(df).mark_point(size=60, opacity=0.7, fill="black", stroke="black").encode(
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / N'),
).properties(
    width=600,
    height=300
)

In [ ]:
hsize_chart = chart + chart.transform_regression('q / p', 'neff / N').mark_line()
hsize_chart

In [ ]:
df = df_fits2[(df_fits2.measure == "degree") & (df_fits2.bounds == "p Fixed")]
chart2 = alt.Chart(df).mark_point(size=60, opacity=0.7, fill="black", stroke="black").encode(
    # x=alt.X('q / p', sort=q_frac_order, axis=None),
    x=alt.X('q / p', sort=q_frac_order),
    # x=alt.X('q', sort=q_frac_order, axis=alt.Axis(labelExpr=q_frac_order)),
    y=alt.Y('neff / Nedges', title="Eeff / E", scale=alt.Scale(domain=[0, 1.1])),
).properties(
    width=600,
    height=300
)

In [ ]:
reg_degree = chart2.transform_regression('q / p', 'neff / Nedges', params=False).mark_line(stroke="black")

In [ ]:
degree_chart = chart2 + reg_degree
degree_chart

In [ ]:
# Faire varier l'axe en 0.10
# appeler Neff et Eeff

degree_chart | hsize_chart
# alt.layer(degree_chart, hsize_chart)

### Regression coeffs

In [ ]:
reg = chart2.transform_regression('q / p', 'neff / Nedges', params=True)
print(altair_transform.extract_data(reg))

In [ ]:
reg2 = chart.transform_regression('q / p', 'neff / N', params=True)
print(altair_transform.extract_data(reg2))

In [ ]:
# Fixer p est plus facilement explicable: plutot parler de ca dans le papier